# **Закрепление знаний**

Этот набор данных создан для того, чтобы на основе определённых диагностических измерений предсказать, есть ли у пациента диабет. 

На выбор этих экземпляров из более крупной базы данных было наложено несколько ограничений. В частности, все пациенты здесь — женщины не моложе 21 года индейского происхождения Пима.

In [45]:
import pandas as pd
from IPython.display import display
import numpy as np
import math
diabetes = pd.read_csv('data/diabetes_data.csv')
display(diabetes.shape[0])
diabetes.head(15)

778

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome,Gender
0,6,98,58,33,190,34.0,0.430,43,0,Female
1,2,112,75,32,0,35.7,0.148,21,0,Female
2,2,108,64,0,0,30.8,0.158,21,0,Female
3,8,107,80,0,0,24.6,0.856,34,0,Female
4,7,136,90,0,0,29.9,0.210,50,0,Female
5,6,103,72,32,190,37.7,0.324,55,0,Female
6,1,71,48,18,76,20.4,0.323,22,0,Female
7,0,117,0,0,0,33.8,0.932,44,0,Female
8,4,154,72,29,126,31.3,0.338,37,0,Female
9,5,147,78,0,0,33.7,0.218,65,0,Female


In [46]:
# Начнём с поиска дубликатов в данных. Найдите все повторяющиеся строки в данных и удалите их.
# Для поиска используйте все признаки в данных. Сколько записей осталось в данных?

dupl_columns = list(diabetes.columns)

mask = diabetes.duplicated(subset=dupl_columns) # Создадим маску дубликатов с помощью метода duplicated() и произведём фильтрацию
diabetes_duplicates = diabetes[mask]
print(f'Число найденных дубликатов: {diabetes_duplicates.shape[0]}')

diabetes = diabetes.drop_duplicates(subset=dupl_columns)
print(f'Результирующее число записей: {diabetes.shape[0]}')

Число найденных дубликатов: 10
Результирующее число записей: 768


In [47]:
# Далее найдите все неинформативные признаки в данных и избавьтесь от них.
# В качестве порога информативности возьмите 0.99: удалите все признаки,
# для которых 99 % значений повторяются или 99 % записей уникальны.
# В ответ запишите имена признаков, которые вы нашли (без кавычек).

#список неинформативных признаков
low_information_cols = []

#цикл по всем столбцам
for col in diabetes.columns:
    #наибольшая относительная частота в признаке
    top_freq = diabetes[col].value_counts(normalize=True).max()
    #доля уникальных значений от размера признака
    nunique_ratio = diabetes[col].nunique() / diabetes[col].count()
    # сравниваем наибольшую частоту с порогом
    if top_freq > 0.95:
        low_information_cols.append(col)
        print(f'{col}: {round(top_freq*100, 2)}% одинаковых значений')
    # сравниваем долю уникальных значений с порогом
    if nunique_ratio > 0.95:
        low_information_cols.append(col)
        print(f'{col}: {round(nunique_ratio*100, 2)}% уникальных значений')
        
diabetes = diabetes.drop(low_information_cols, axis=1)
display(diabetes.shape[1])

Gender: 100.0% одинаковых значений


9

In [48]:
# Замените все записи, равные 0, в столбцах Glucose, BloodPressure, SkinThickness,
# Insulin и BMI на символ пропуска. Его вы можете взять из библиотеки numpy: np.nan.
# Какая доля пропусков содержится в столбце Insulin? Ответ округлите до сотых.

diabetes['Glucose']=diabetes['Glucose'].apply(lambda x: x if x!=0 else np.nan)
diabetes['BloodPressure']=diabetes['BloodPressure'].apply(lambda x: x if x!=0 else np.nan)
diabetes['SkinThickness']=diabetes['SkinThickness'].apply(lambda x: x if x!=0 else np.nan)
diabetes['Insulin']=diabetes['Insulin'].apply(lambda x: x if x!=0 else np.nan)
diabetes['BMI']=diabetes['BMI'].apply(lambda x: x if x!=0 else np.nan)

display(diabetes['Insulin'].value_counts(normalize=True,dropna=False))

NaN      0.486979
105.0    0.014323
140.0    0.011719
130.0    0.011719
120.0    0.010417
           ...   
485.0    0.001302
370.0    0.001302
342.0    0.001302
65.0     0.001302
235.0    0.001302
Name: Insulin, Length: 186, dtype: float64

In [49]:
# Удалите из данных признаки, где число пропусков составляет более 30 %.
# Сколько признаков осталось в ваших данных (с учетом удаленных неинформативных признаков в задании 8.2)?

# отбрасываем столбцы с числом пропусков более 30% (100-30)
n = diabetes.shape[0] # число строк в таблице
thresh = math.ceil(n*0.7)
diabetes = diabetes.dropna(how='any', thresh=thresh, axis=1)
display(diabetes.shape[1])

8

In [50]:
# Удалите из данных только те строки, в которых содержится более двух пропусков одновременно.
# Чему равно результирующее число записей в таблице

# отбрасываем строки с числом пропусков более 2 в строке
cols = diabetes.shape[1] #число признаков после удаления столбцов
diabetes = diabetes.dropna(how='any', thresh=cols-2, axis=0)
display(diabetes.shape[0])

761

In [58]:
# В оставшихся записях замените пропуски на медиану.
# Чему равно среднее значение в столбце SkinThickness? Ответ округлите до десятых.
diabetes=diabetes.fillna(value={'Glucose': diabetes['Glucose'].median(), 'BloodPressure': diabetes['BloodPressure'].median(), 'SkinThickness': diabetes['SkinThickness'].median(), 'BMI': diabetes['BMI'].median()}) 
display(diabetes['SkinThickness'].mean().round(1))

29.1

In [72]:
# Сколько выбросов найдёт классический метод межквартильного размаха в признаке SkinThickness?

def outliers_iqr_mod(data, feature, left=1.5, right=1.5, log_scale=False):
    if log_scale==False:
        x = data[feature]
    else:
        x = np.log(data[feature])
    quartile_1, quartile_3 = x.quantile(0.25), x.quantile(0.75),
    iqr = quartile_3 - quartile_1
    lower_bound = quartile_1 - (iqr * left)
    upper_bound = quartile_3 + (iqr * right)
    outliers = data[(x<lower_bound) | (x > upper_bound)]
    cleaned = data[(x>lower_bound) & (x < upper_bound)]
    return outliers, cleaned

outliers, cleaned = outliers_iqr_mod(diabetes, 'SkinThickness')
print(f'Число выбросов по методу Тьюки: {outliers.shape[0]}')
print(f'Результирующее число записей: {cleaned.shape[0]}')


Число выбросов по методу Тьюки: 87
Результирующее число записей: 674


In [63]:
# Сколько выбросов найдёт классический метод z-отклонения в признаке SkinThickness?

def outliers_z_score(data, feature, log_scale=False):
    if log_scale:
        x = np.log(data[feature]+1)
    else:
        x = data[feature]
    mu = x.mean()
    sigma = x.std()
    lower_bound = mu - 3 * sigma
    upper_bound = mu + 3 * sigma
    outliers = data[(x < lower_bound) | (x > upper_bound)]
    cleaned = data[(x > lower_bound) & (x < upper_bound)]
    return outliers, cleaned

outliers, cleaned = outliers_z_score(diabetes, 'SkinThickness')
print(f'Число выбросов по методу z-отклонения: {outliers.shape[0]}')
print(f'Результирующее число записей: {cleaned.shape[0]}')

Число выбросов по методу z-отклонения: 4
Результирующее число записей: 757


In [73]:
# Найдите сначала число выбросов в признаке DiabetesPedigreeFunction с помощью классического метода межквартильного размаха.
# Затем найдите число выбросов в этом же признаке в логарифмическом масштабе (при логарифмировании единицу прибавлять не нужно!).
# Какова разница между двумя этими числами

outliers, cleaned = outliers_iqr_mod(diabetes, 'DiabetesPedigreeFunction')
print(f'Число выбросов по методу Тьюки: {outliers.shape[0]}')
print(f'Результирующее число записей: {cleaned.shape[0]}')

outliers_LOG, cleaned_LOG = outliers_iqr_mod(diabetes, 'DiabetesPedigreeFunction',log_scale=True)
print(f'Число выбросов по методу Тьюки с логарифмированием: {outliers_LOG.shape[0]}')
print(f'Результирующее число записей: {cleaned_LOG.shape[0]}')

difference = outliers.shape[0] - outliers_LOG.shape[0]
display(difference)

Число выбросов по методу Тьюки: 29
Результирующее число записей: 732
Число выбросов по методу Тьюки с логарифмированием: 0
Результирующее число записей: 761


29